## Visualisation des données Air Pays de la Loire

Accèssibles sur leur [site](https://www.airpl.org/), les données mesurées concernent de nombreux polluants atmosphériques à des points précis de mesures des taux de concentration. 

Pour une première analyse, nous nous interessons aux concentrations en PM10 dans le departement Loire-Atlantique. Notre pas de mesure est d'une heure, et pour limiter le poids des données, nous nous concentrons sur une semaine (du 19/03/2023 au 25/03/2023)

D'abord, chargeons les données et observons-les

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import geopandas as gpd
import plotly.express as px
from pynsee.geodata import get_geodata_list, get_geodata

In [2]:
raw_df_PM10_hour_week = pd.read_csv(r'./PM10_horaire_semaine.csv', sep=';')
raw_df_PM10_hour_week.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1848 entries, 0 to 1847
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   nom_dept           1848 non-null   object 
 1   nom_com            1848 non-null   object 
 2   insee_com          1848 non-null   int64  
 3   nom_station        1848 non-null   object 
 4   code_station (ue)  1848 non-null   object 
 5   influence          1848 non-null   object 
 6   nom_poll           1848 non-null   object 
 7   id_poll_ue         1848 non-null   int64  
 8   valeur             1834 non-null   float64
 9   unite              1848 non-null   object 
 10  metrique           1848 non-null   object 
 11  date_debut         1848 non-null   object 
 12  date_fin           1848 non-null   object 
 13  statut_valid       1848 non-null   bool   
 14  x_wgs84            1848 non-null   float64
 15  y_wgs84            1848 non-null   float64
 16  x_reglementaire    1848 

On observe 18 colonnes décrivant nos data : 

- nom_dept : nom du département où se situe la station de mesure (issu des données de l’IGN).

- nom_com : nom de la commune où se situe la station de mesure (issu des données de l’IGN).

- insee_com : code INSEE de la commune où se situe la station de mesure (issu des données de l’IGN).

- nom_station : nom de la station de mesure déterminé par Air Pays de la Loire.

- code_station (ue) : code unique de la station de mesure.

- influence : nature de la station de mesure déterminée par sa localisation et son influence selon le type de polluant qu’elle mesure. 

- nom_poll : nom du polluant mesuré par la station de mesure.

- id_poll_ue : identifiant du polluant dans le référentiel européen.

- valeur : valeur de la mesure relevée pour un polluant à une station et à une métrique données.

- unite : unité de la valeur mesurée du polluant.

- metrique : niveau d’agrégation temporelle des données mesurées (horaire, journalière, mensuelle, annuelle).

- date_debut : date de début du relevé de mesure en heure temps local (heure sur le territoire métropolitain français).

- date_fin : date de fin du relevé de mesure en heure temps local (heure sur le territoire métropolitain français).

- statut_valid : validité de la mesure relevée, « t » si la mesure est validée, « f » si la mesure est invalidée.

- x_wgs84 : coordonnée x de la localisation de la station de mesure en World Geodetic System 1984 (ESPG : 4326)

- y_wgs84 : coordonnée y de la localisation de la station de mesure en World Geodesic System 1984 (ESPG : 4326)

- x_reglementaire : coordonnée x de la localisation de la station de mesure en Lambert 93 (EPSG : 2154).

- y_reglementaire : coordonnée y de la localisation de la station de mesure en Lambert 93 (EPSG : 2154).

In [3]:
raw_df_PM10_hour_week.head()

,nom_dept,nom_com,insee_com,nom_station,code_station (ue),influence,nom_poll,id_poll_ue,valeur,unite,metrique,date_debut,date_fin,statut_valid,x_wgs84,y_wgs84,x_reglementaire,y_reglementaire
0,Loire-Atlantique,Donges,44052,LA MEGRETAIS,FR23003,industrial,PM10,5,6.7,µg/m3,horaire,2023-03-25 22:00:00,2023-03-25 23:00:00,True,-2.039245,47.322705,319631.265310,6.703517e+06
1,Loire-Atlantique,Frossay,44061,FROSSAY,FR23068,industrial,PM10,5,2.7,µg/m3,horaire,2023-03-25 22:00:00,2023-03-25 23:00:00,True,-1.936524,47.247352,326851.682361,6.694673e+06
2,Loire-Atlantique,Saint-Etienne-De-Montluc,44158,SAINT ETIENNE DE MONTLUC,FR23070,industrial,PM10,5,11.0,µg/m3,horaire,2023-03-25 22:00:00,2023-03-25 23:00:00,True,-1.783081,47.283127,338676.423909,6.697926e+06
3,Loire-Atlantique,Nantes,44109,LA CHAUVINIERE,FR23107,background,PM10,5,5.7,µg/m3,horaire,2023-03-25 22:00:00,2023-03-25 23:00:00,True,-1.573275,47.252912,354313.235770,6.693636e+06
4,Loire-Atlantique,Saint-Nazaire,44184,LEON BLUM,FR23110,background,PM10,5,13.0,µg/m3,horaire,2023-03-25 22:00:00,2023-03-25 23:00:00,True,-2.250154,47.264110,303302.164390,6.698060e+06


# Preprocessing : 
On veut épurer le dataframe des colonnes qui ne nous interessent pas, et renommer certaines pour plus de facilité dans la manipulation

In [4]:
col_of_interess = ['nom_station', 'code_station (ue)', 'valeur', 'date_fin', 'statut_valid', 'x_wgs84', 'y_wgs84']
df_PM10_hour_week = raw_df_PM10_hour_week[col_of_interess]
df_PM10_hour_week = df_PM10_hour_week.rename(columns={'code_station (ue)': 'code_station', 'date_fin': 'heure_mesure', 'x_wgs84': 'lon', 'y_wgs84': 'lat'})
df_PM10_hour_week['heure_mesure'] = pd.to_datetime(df_PM10_hour_week['heure_mesure'])
df_PM10_hour_week.head()

,nom_station,code_station,valeur,heure_mesure,statut_valid,lon,lat
0,LA MEGRETAIS,FR23003,6.7,2023-03-25 23:00:00,True,-2.039245,47.322705
1,FROSSAY,FR23068,2.7,2023-03-25 23:00:00,True,-1.936524,47.247352
2,SAINT ETIENNE DE MONTLUC,FR23070,11.0,2023-03-25 23:00:00,True,-1.783081,47.283127
3,LA CHAUVINIERE,FR23107,5.7,2023-03-25 23:00:00,True,-1.573275,47.252912
4,LEON BLUM,FR23110,13.0,2023-03-25 23:00:00,True,-2.250154,47.264110


In [5]:
# y a t-il des lignes invalides ? 
invalid = df_PM10_hour_week['statut_valid'].value_counts()
print(invalid)

# TODO : les filtrer

statut_valid
True     1834
False      14
Name: count, dtype: int64


# Visualisation simple de chaque station sur la journée du 25/03/2023

Pour se familiariser avec les données récupérées, nous allons tracer l'évolution heure par heure des concentrations en PM10 sur la journée du 25/03/2023 des 11 stations relevées.

In [6]:
df_PM10_25032023 = df_PM10_hour_week.loc[df_PM10_hour_week['heure_mesure'].dt.date == pd.to_datetime('2023-03-25').date()]

px.line(df_PM10_25032023,
        x='heure_mesure',
        y='valeur',
        color='nom_station',
        title='Evolution des valeurs de PM10 sur la journée du 25/03/2023 en Loire-Atlantique')


On observe comme prévu une augmentation de la concentration de PM10 pendant la journée, et une moindre concentration la nuit. 
La mesure des polluants dans l'air est lié à beaucoup de phénomène, et les évolutions (dont pics) peuvent s'expliquer par les conditions météo ou une intervention humaine particulière. 

Mais d'abord, où sont situées nos stations ? En les affichant avec plotly, nous allons observer leur valeur moyenne en PM10

In [9]:
df_localisation = df_PM10_hour_week[['nom_station', 'lon', 'lat', 'valeur']]
df_localisation = df_localisation.groupby(['nom_station']).agg({'lon': 'first', 'lat': 'first', 'valeur': 'mean'})
df_localisation.reset_index(inplace=True)

fig = px.scatter_mapbox(df_localisation,
                        lat='lat',
                        lon='lon', 
                        hover_name='nom_station',
                        color='valeur',
                        size='valeur',
                        zoom=9,
                        height=400,
                        size_max=10,
                        opacity=0.9,
                        width=800
                        )
fig.update_layout(mapbox_style='stamen-terrain')
fig.update_layout(margin={"r": 0, "t": 40, "l": 0, "b": 0})
fig.update_layout(title_text="Localisation des stations avec leur valeur moyenne en PM10 sur la semaine du 19 au 25 mars 2023")
fig.show()

Il est interessant de suivre sur la semaine l'evolution de stations géographiquement proches : Trentemoult et les Couets. 

In [ ]:
df_PM10_Trent_Couets = df_PM10_hour_week.loc[(df_PM10_hour_week['nom_station'] == 'LES COUETS') | (df_PM10_hour_week['nom_station'] == 'TRENTEMOULT')]

px.line(df_PM10_Trent_Couets,
        x='heure_mesure',
        y='valeur',
        color='nom_station',
        title='Evolution des valeurs de PM10 sur la semaine du 19/03/2023 au 25/03/2023 en Loire-Atlantique')

Les deux stations ont une évolution similaire, mais on peut noter quelques pics spcifiques : 
- le 20 mars entre 20h et 22h pour Trentemoult
- le 21 mars entre 15h et 17h pour les Couets

# TODO

Ce que l'on peut prévoir pour approfondir : 

- ajouter d'autres graphiques (cumule des émissions, émissions moyennes...)

- se focaliser sur une stations et ajouter cartographiquement les points d'émissions (routes, résidences...) 